In [2]:
import torch
import pandas as pd
import numpy as np
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [2]:
device = "cpu"

In [3]:
df = pd.read_csv('data.csv', sep=";")
df.head()

,Player1,Player2,Player3,Player4,Eichel Ober,Eichel Unter,Eichel Ass,Eichel 10,Eichel König,Eichel 9,...,SchellenOber,SchellenUnter,SchellenAss,Schellen10,SchellenKönig,Schellen9,Schellen8,Schellen7,Modus,Win
0,1,0,0,0,1,0,1,0,1,0,...,0,0,0,0,1,1,0,0,weiter,-1
1,0,1,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,weiter,-1
2,0,0,0,1,0,1,0,0,0,0,...,1,1,0,0,0,0,0,1,Farbwenz Schelle,1
3,0,0,0,1,1,0,0,0,0,0,...,1,1,0,0,0,0,0,1,Farbgeier Schelle,1
4,1,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,1,1,weiter,1


In [4]:
df_filtered = df[df['Modus'] != 'weiter']
game_modes = df_filtered['Modus'].unique()
df_filtered['Modus'].value_counts(normalize=True)


Sauspiel Alte            0.172273
Sauspiel Hundsgfickte    0.166309
Sauspiel Blaue           0.163832
Geier                    0.051654
Farbwenz Gras            0.037755
Farbgeier Gras           0.037755
Farbwenz Herz            0.037603
Farbgeier Herz           0.037603
Farbwenz Schelle         0.037073
Farbgeier Schelle        0.037073
Farbgeier Eichel         0.036239
Farbwenz Eichel          0.036239
Wenz                     0.036011
Solo Gras                0.029163
Solo Herz                0.029112
Solo Schelle             0.027545
Solo Eichel              0.026762
Name: Modus, dtype: float64

In [5]:
train_data = df_filtered.drop(['Player1','Player2','Player3','Player4','Win','Modus'], axis=1).astype("float32")
train_data.head()

,Eichel Ober,Eichel Unter,Eichel Ass,Eichel 10,Eichel König,Eichel 9,Eichel 8,Eichel 7,Gras Ober,Gras Unter,...,Herz 8,Herz 7,SchellenOber,SchellenUnter,SchellenAss,Schellen10,SchellenKönig,Schellen9,Schellen8,Schellen7
2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
7,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [6]:
train_data_size = np.shape(np.array(train_data))
input_size = train_data_size[1]
print("Input Size: ", input_size)

Input Size:  32


In [7]:
output_data = df_filtered['Modus']
output_data_codes = output_data.astype("category").cat.codes
output_size = max(output_data_codes) + 1
y = output_data_codes.astype("long")
print("Output Size: ", output_size)

Output Size:  17


In [8]:
X_train, X_val, y_train, y_val = train_test_split(train_data, y, test_size=0.2, random_state=42)
y_train.head()

103571    11
114923     8
127855     5
51701     11
88413     10
dtype: int64

In [9]:
class SelectGameDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.tensor([self.X.iloc[idx]], device=device), torch.tensor(self.y.iloc[idx], device=device)

In [10]:
train_dataset = SelectGameDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [11]:
test_dataset = SelectGameDataset(X_val, y_val)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [12]:
class SelectGameNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, output_size)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [13]:
model = SelectGameNN().to(device)
print(model)

SelectGameNN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=32, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=17, bias=True)
  )
)


In [14]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [15]:
criterion = nn.CrossEntropyLoss()

In [16]:
num_epochs = 300
print_freq = 250  

for epoch in range(num_epochs):
    running_loss = 0.0  
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()  
        optimizer.step()
        
        
        running_loss += loss.item()

        if (batch_idx + 1) % print_freq == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}] Batch [{batch_idx+1}/{len(train_loader)}] Loss: {running_loss / print_freq:.4f}")
            running_loss = 0.0  

Epoch [1/300] Batch [250/495] Loss: 2.1356
Epoch [2/300] Batch [250/495] Loss: 1.1544
Epoch [3/300] Batch [250/495] Loss: 0.9880
Epoch [4/300] Batch [250/495] Loss: 0.8728
Epoch [5/300] Batch [250/495] Loss: 0.8332
Epoch [6/300] Batch [250/495] Loss: 0.8015
Epoch [7/300] Batch [250/495] Loss: 0.7824
Epoch [8/300] Batch [250/495] Loss: 0.7697
Epoch [9/300] Batch [250/495] Loss: 0.7608
Epoch [10/300] Batch [250/495] Loss: 0.7539
Epoch [11/300] Batch [250/495] Loss: 0.7367
Epoch [12/300] Batch [250/495] Loss: 0.7299
Epoch [13/300] Batch [250/495] Loss: 0.7280
Epoch [14/300] Batch [250/495] Loss: 0.7182
Epoch [15/300] Batch [250/495] Loss: 0.7095
Epoch [16/300] Batch [250/495] Loss: 0.6949
Epoch [17/300] Batch [250/495] Loss: 0.6888
Epoch [18/300] Batch [250/495] Loss: 0.6866
Epoch [19/300] Batch [250/495] Loss: 0.6893
Epoch [20/300] Batch [250/495] Loss: 0.6840
Epoch [21/300] Batch [250/495] Loss: 0.6799
Epoch [22/300] Batch [250/495] Loss: 0.6816
Epoch [23/300] Batch [250/495] Loss: 0.67

In [3]:
torch.save(model.state_dict(),'game_classifier.pth')

In [ ]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

# TODO: Evaluate for seperate classes

model.train()